# Bonus Project individual

## SQP Method

### Normal SQP

for each problem choose 5 starting points

In [2]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [3]:
def gradient(f, eps):
    def g(x_vec):
        n = len(x_vec)
        grad = np.zeros(n)
        x = np.copy(x_vec)
        e = np.eye(n)

        # based on equation 8.7 from the book
        for i in range(n):
            grad[i] = (f(x + e[i]*eps) - f(x-e[i]*eps)) / (2 * eps)
        return grad
    return g

def gradient_2(f, eps):
    def g(x_vec):
        n = len(x_vec)
        hessian = np.zeros((n, n))
        x = np.copy(x_vec)
        e = np.eye(n)

        # based on equation 8.21 from the book
        for i in range(n):
            for j in range(n):
                hessian[i][j] = (f(x + e[i]*eps + e[j]*eps) - f(x+e[i]*eps) - f(x+e[j]*eps) + (f(x))) / eps**2
        return hessian
    return g

In [4]:
def SQP_Method(x_0):
    x = x_0
    n_iterations = 0
    return x, n_iterations 

### 1)
$$\min e^{x_1 x_2 x_3 x_4 x_5} − {1\over 2} (x_1^3 + x_2^3 + 1)^2$$
$$\text{subject to } x_1^2 + x_2^ 2 + x_3 ^2 + x_4 ^2 + x_5 ^2 − 10 = 0$$
$$x_2 x_3 − 5x_4 x_5 = 0$$
$$x_1^3 + x_2^3 + 1 = 0$$
starting points $x_0 = (−1.71, 1.59, 1.82, −0.763, −0.763)^T$  
The solution is $x^*=(−1.8, 1.7, 1.9, −0.8, −0.8)^T$

Additional starting points: $x_0 = (0,0,0,0,0)^T$, $x_0 = (1,0,3,0,0)^T$, $x_0 = (3,0,1,1,0)^T$,  $x_0 = (0,1,2,3,4)^T$  

In [5]:
def f(x):
    return np.exp(x[0]*x[1]*x[2]*x[3]*x[4]) - 0.5 * (x[0]**3 + x[1]**3 + 1)**2

# equality constraints
def eqc_1(x):
    return x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 -10

def eqc_2(x):
    return x[1] * x[2] - 5 * x[3] * x[4]

def eqc_3(x):
    return x[0]**3 + x[1]**3 + 1

starting_points = [np.array([-1.71, 1.59, 1.82, -0.763, -0.763]),
                   np.array([0.0, 0.0, 0.0, 0.0, 0.0]),
                   np.array([1.0, 0.0, 3.0, 0.0, 0.0]),
                   np.array([3.0, 0.0, 1.0, 1.0, 0.0]),
                   np.array([0.0, 1.0, 2.0, 3.0, 4.0])]

In [9]:
# scipy libary
for x0 in starting_points:
    bounds = [(None, None), (None, None), (None, None), (None, None), (None, None)]
    constraint_definition = [{'type': 'eq', 'fun': eqc_1},
                             {'type': 'eq', 'fun': eqc_2},
                             {'type': 'eq', 'fun': eqc_3}]

    result = minimize(f, x0, method='SLSQP', bounds=bounds, constraints=constraint_definition)

    print(result)
    print()

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.053949847760167556
       x: [-1.717e+00  1.596e+00  1.827e+00 -7.636e-01 -7.636e-01]
     nit: 3
     jac: [ 9.173e-02 -9.871e-02 -8.620e-02  2.063e-01  2.063e-01]
    nfev: 18
    njev: 3

 message: Singular matrix C in LSQ subproblem
 success: False
  status: 6
     fun: 0.5
       x: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
     nit: 1
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
    nfev: 6
    njev: 1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.0
       x: [-1.000e+00 -2.968e-32  3.000e+00  0.000e+00  0.000e+00]
     nit: 8
     jac: [ 3.813e-04  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
    nfev: 49
    njev: 8

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.0
       x: [-1.000e+00  8.820e-14  2.121e+00  2.121e+00  1.764e-14]
     nit: 9
     jac: [ 1.040e-04  0.000e+00  0.0

/tmp/ipykernel_39633/3008426413.py:2: RuntimeWarning: overflow encountered in exp
  return np.exp(x[0]*x[1]*x[2]*x[3]*x[4]) - 0.5 * (x[0]**3 + x[1]**3 + 1)**2


### 2)

Rosenbrock function with the following constraints:  
$x_1^2+ x_2^2 \le 1$  
$x_2 \ge 0$

with the starting points  
$x_0 = (0.8,0.6)^T$, $x_0 = (1,0)^T$, $x_0 = (1,1)^T$, $x_0 = (0,0)^T$, $x_0 = (-1,1)^T$  

In [ ]:
def f(x):
    return 100*((x[2]-x[1]**2)**2)+(1-x[1])**2

starting_points = [np.array([0.8, 0.6]),
                   np.array([1.0, 0.0]),
                   np.array([1.0, 1.0]),
                   np.array([0.0, 0.0]),
                   np.array([-1.0, 1.0])]

### 3)
The function
$$f (x) = 150(x_1x_2)^2 + (0.5x_1 + 2x_2 − 2)^2 ,$$

constraints
$$0 ≤ c_1 (x) = (x_1 − 1/2)2 + (x_2 − 1)2 − 5/16$$
$$0 ≤ c_2 (x) = (x_1 + 1)2 + (x_2 − 3/8)2 − 73/64$$
$$0 ≤ c_3 (x) = −(x_1 + 1)2 − (x_2 − 1)2 + \sqrt 2$$

In [ ]:
def f(x):
    return 150*(x[1]*x[2])**2+(0.5*x[1]+2*x[2]-2)**2

starting_points = [np.array([0.1, 0.74]),
                   np.array([0.0, 0.0]),
                   np.array([0.5, 0.5]),
                   np.array([0.0, 0.76]),
                   np.array([-0.2, 0.8])]

### 4) 

constraints: $$\|x\|^2_2 \le 1$$

## Warm Start or Full Quasi-Newton approximation